In [1]:
code="""
#include <iostream>
#include <bits/stdc++.h>
using namespace std;
__global__ void mulGPU(int *a,int *b,int *c,int size){
  int row=blockIdx.y*blockDim.y+threadIdx.y;
  int col=blockIdx.x*blockDim.x+threadIdx.x;
  if(row<size &&col<size){
    int sum=0;
    for(int k=0;k<size;k++){
      sum+=a[row*size+k]*b[k*size+col];
    }
    c[row*size+col]=sum;
  }
}
void printMatrix(vector<int>a,int size){
  for(int i=0;i<size;i++){
    for(int j=0;j<size;j++){
      cout<<a[i*size+j]<<" ";
    }
    cout<<endl;
  }
  cout<<endl;
}
int main(){
  int size=3;
  vector<int>a={
    1,2,3,
    4,5,6,
    7,8,9
  };
  vector<int>b={
    9,8,7,
    6,5,4,
    3,2,1
  };
  vector<int>c(size*size);
  printMatrix(a,3);
  printMatrix(b,3);

  int *da, *db,*dc;
  int totalbytes=size*size*sizeof(int);
  cudaMalloc(&da,totalbytes);
    cudaMalloc(&db,totalbytes);
  cudaMalloc(&dc,totalbytes);

  cudaMemcpy(da,a.data(),totalbytes,cudaMemcpyHostToDevice);
  cudaMemcpy(db,b.data(),totalbytes,cudaMemcpyHostToDevice);

  dim3 threadsperblock(16,16);
  dim3 blockspergrid((size+15)/16,(size+15)/16);
mulGPU<<<blockspergrid,threadsperblock>>>(da,db,dc,size);
cudaGetLastError();
cudaDeviceSynchronize();
cudaMemcpy(c.data(),dc,totalbytes,cudaMemcpyDeviceToHost);
printMatrix(c,3);
cudaFree(da);
cudaFree(db);
cudaFree(dc);
  return 0;

}
"""
with open("mul.cu","w") as f:
  f.write(code)
!nvcc -arch=sm_75 mul.cu -o mul
!./mul

1 2 3 
4 5 6 
7 8 9 

9 8 7 
6 5 4 
3 2 1 

30 24 18 
84 69 54 
138 114 90 

